In [1]:
# import dependencies

import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup 
from collections import defaultdict
import time
import random
import spotipy


In [50]:
# Fix the constant portion of the url

url = 'https://spotifycharts.com/viral/sg/daily/'

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
song_dict = defaultdict(int)

for each_track in soup.find_all(class_ = 'chart-table-track'):
        title = each_track.find('strong')
        artist = each_track.find('span')
        if title != None or artist != None:
            song_dict[(title.text, artist.text.replace("by", ""))] += 1
            

In [57]:
# Collect all info at once

for each_track in soup.find_all('tr')[1:]:
    title = each_track.find('strong').text
    artist = each_track.find('span').text.replace("by", "")
    track_id = each_track.find('a')['href'][31:]
    print(title, artist, track_id)

hot girl bummer  blackbear 7aiClxsDWFRQ0Kzk5KI5ku
Haa Tepok  MeerFly 65pndcjVsNEJpwxfTgcdv0
Maya  Mugen Rao 3Um4k1xWbxabFjwJAs8vdd
說好不哭  Jay Chou 56wVfJKtnwlSZtC4NVgIrf
รักติดไซเรน  แพรวา ณิชาภัทร 0vIvE2RgjnMpUQn875C4Vz
像魚  王貳浪 1MRRjWwqBUyoD9dC11sZ4S
STUPID (feat. Yung Baby Tate)  Ashnikko 6qNB2ChCVPepl5ZjVJJTUW
Rock And Roll Part II  Gary Glitter 63W11KVHDOpSlh3XMQ7qMg
芒种  音阙诗听 3IP4qSMfGeqI2XqHFf25sj
Dance Monkey  Tones and I 1rgnBhdG2JDFTbYkYRZAku
Come Callin'  Astronauts 4zaXq3MbMooGgwui6uA6kk
Salah Apa Aku  Ilir7 7IlCcQ8bYUEMwC2SDHgZ5b
Smile (From United Artists Film "Modern Times")  Jimmy Durante 7M18i7KC9A1mTrQVPY9hKy
Valentino  24kGoldn 0c7K73roy6RW4ebDkf4yGh
Way Maker - Live  Leeland 40ozK0lCJ0EQzVGWsCNUhk
Ride It  Regard 2tnVG71enUj33Ic2nFN6kZ
Sorang  Luqman Podolski 2e3J1dAm54o9YcZVFo6nl8
Waiting For The Sun  Bars and Melody 7FnsByHEDrojOvrssSM9Lw
Gordon Ramsay  HL Wave 78uUSmxaoXEBf9gFKFYK3y
Sunkissed  khai dreams 4efZXKOFm1rk22FDjScLbE
Sin City  Chrishan 28SYU45WvN59hVOdvQp

In [5]:
# Define range of variable

end_date = dt.datetime.strptime("2019-10-12", "%Y-%m-%d")
start_date = dt.datetime.strptime("2019-01-01", "%Y-%m-%d")
date_values = [start_date + dt.timedelta(days = x) for x in range(0, (end_date-start_date).days)]
full_version = lambda x: url+x


In [6]:
for each_value in date_values:
    #print(full_url(each_value.date().__str__()))
    print(each_value.date())

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01

In [183]:
def spotify_viral(start_date, end_date):
    '''
    inputs (str): scrapes tracks from Spotify viral charts by 
    takeing in start_date and end_date with format YYYY-MM-DD
    
    
    outputs (dict): returns a dictonary with
    each combination of 'track_id', 'title', and 'artist' as a key; and
    no. of times the track appears in top 50 viral chart
    as value for the period.
    '''
    
    base_url = 'https://spotifycharts.com/viral/sg/daily/'
    
    end_date = dt.datetime.strptime(end_date, "%Y-%m-%d")
    start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
    
    dates = [start_date + dt.timedelta(days = x) for x in range(0, 1+(end_date-start_date).days)]

    full_version = lambda x: url+x

    song_dict = defaultdict(int)
    
    for each_date in dates:
        print(each_date)
        full_url = base_url + each_date.date().__str__()
        response = requests.get(full_url)
        page = response.text
        soup = BeautifulSoup(page, 'html.parser')
        
        for each_track in soup.find_all('tr')[1:]:
            title = each_track.find('strong').text
            artist = each_track.find('span').text.replace("by", "")
            track_id = each_track.find('a')['href'][31:]
            song_dict[(track_id, title, artist)] += 1

        
        time.sleep(random.randint(1,6))
        
    return song_dict

In [73]:
# Get no. of times a track has been on viral chart for a given period 

song_dict = spotify_viral(start_date = '2019-10-02', end_date = '2019-10-02')

In [74]:
print(song_dict)


defaultdict(<class 'int'>, {('65pndcjVsNEJpwxfTgcdv0', 'Haa Tepok', ' MeerFly'): 1, ('1MRRjWwqBUyoD9dC11sZ4S', '像魚', ' 王貳浪'): 1, ('7aiClxsDWFRQ0Kzk5KI5ku', 'hot girl bummer', ' blackbear'): 1, ('3IP4qSMfGeqI2XqHFf25sj', '芒种', ' 音阙诗听'): 1, ('5uWDUe75h2FHLZyMX9r5sp', 'Sunkissed', ' khai dreams'): 1, ('56pvqFKGXPjubh5eY6sOlv', 'Is There Still Anything That Love Can Do?', ' RADWIMPS'): 1, ('2cE5PbW9PhPmnyETXqH3VE', 'Grand Escape (Movie Edit) [feat. Toko Miura]', ' RADWIMPS'): 1, ('0TMGV2zMZXOILVtQLtkI3G', 'What Am I', " Why Don't We"): 1, ('1rgnBhdG2JDFTbYkYRZAku', 'Dance Monkey', ' Tones and I'): 1, ('2tnVG71enUj33Ic2nFN6kZ', 'Ride It', ' Regard'): 1, ('0SIAFU49FFHwR3QnT5Jx0k', 'Chinese New Year', ' SALES'): 1, ('21jGcNKet2qwijlDFuPiPb', 'Circles', ' Post Malone'): 1, ('40ozK0lCJ0EQzVGWsCNUhk', 'Way Maker - Live', ' Leeland'): 1, ('2e3J1dAm54o9YcZVFo6nl8', 'Sorang', ' Luqman Podolski'): 1, ('0c7K73roy6RW4ebDkf4yGh', 'Valentino', ' 24kGoldn'): 1, ('0vIvE2RgjnMpUQn875C4Vz', 'รักติดไซเรน', '

In [ ]:
# Use spotify API

with open('/home/thedatacurious/Documents/Metis/spotify.txt') as f:
    content = f.readlines()
    content = [x.strip() for x in content]

client_id = content[0]
secret = content[1]

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [158]:
def extract_features(song_dict):
    '''
    inputs (dict): takes in dictionary 'track_id', 'title', and 'artist' key 
    use 'track_id' to call spotify api for attributes
    
    outputs (dataframe): returns a dataframe with track attributes as columns
    '''
    
    counter = 0
    tracks_list_wfeatures = list()

    for key in song_dict.keys():
        counter += 1
        track_id = key[0]
        a_track_features = sp.audio_features(track_id)[0]
        tracks_list_wfeatures.append(a_track_features)
        if counter % 25 == 0:
            time.sleep(random.randint(1,6))

    df = pd.DataFrame(tracks_list_wfeatures)
    
    return df

In [181]:
df = pd.DataFrame(song_dict.items(), columns = ['tup', 'chart_appearances'])
df[['track_id', 'title', 'artist']] = pd.DataFrame(df.iloc[:,0].tolist())
df

,tup,chart_appearances,track_id,title,artist
0,"(65pndcjVsNEJpwxfTgcdv0, Haa Tepok, MeerFly)",1,65pndcjVsNEJpwxfTgcdv0,Haa Tepok,MeerFly
1,"(1MRRjWwqBUyoD9dC11sZ4S, 像魚, 王貳浪)",1,1MRRjWwqBUyoD9dC11sZ4S,像魚,王貳浪
2,"(7aiClxsDWFRQ0Kzk5KI5ku, hot girl bummer, bla...",1,7aiClxsDWFRQ0Kzk5KI5ku,hot girl bummer,blackbear
3,"(3IP4qSMfGeqI2XqHFf25sj, 芒种, 音阙诗听)",1,3IP4qSMfGeqI2XqHFf25sj,芒种,音阙诗听
4,"(5uWDUe75h2FHLZyMX9r5sp, Sunkissed, khai dreams)",1,5uWDUe75h2FHLZyMX9r5sp,Sunkissed,khai dreams
5,"(56pvqFKGXPjubh5eY6sOlv, Is There Still Anythi...",1,56pvqFKGXPjubh5eY6sOlv,Is There Still Anything That Love Can Do?,RADWIMPS
6,"(2cE5PbW9PhPmnyETXqH3VE, Grand Escape (Movie E...",1,2cE5PbW9PhPmnyETXqH3VE,Grand Escape (Movie Edit) [feat. Toko Miura],RADWIMPS
7,"(0TMGV2zMZXOILVtQLtkI3G, What Am I, Why Don't...",1,0TMGV2zMZXOILVtQLtkI3G,What Am I,Why Don't We
8,"(1rgnBhdG2JDFTbYkYRZAku, Dance Monkey, Tones ...",1,1rgnBhdG2JDFTbYkYRZAku,Dance Monkey,Tones and I
9,"(2tnVG71enUj33Ic2nFN6kZ, Ride It, Regard)",1,2tnVG71enUj33Ic2nFN6kZ,Ride It,Regard


In [171]:
df.drop(columns = 'tup', inplace = True)
df

,chart_appearances,track_id,title,artist
0,1,65pndcjVsNEJpwxfTgcdv0,Haa Tepok,MeerFly
1,1,1MRRjWwqBUyoD9dC11sZ4S,像魚,王貳浪
2,1,7aiClxsDWFRQ0Kzk5KI5ku,hot girl bummer,blackbear
3,1,3IP4qSMfGeqI2XqHFf25sj,芒种,音阙诗听
4,1,5uWDUe75h2FHLZyMX9r5sp,Sunkissed,khai dreams
5,1,56pvqFKGXPjubh5eY6sOlv,Is There Still Anything That Love Can Do?,RADWIMPS
6,1,2cE5PbW9PhPmnyETXqH3VE,Grand Escape (Movie Edit) [feat. Toko Miura],RADWIMPS
7,1,0TMGV2zMZXOILVtQLtkI3G,What Am I,Why Don't We
8,1,1rgnBhdG2JDFTbYkYRZAku,Dance Monkey,Tones and I
9,1,2tnVG71enUj33Ic2nFN6kZ,Ride It,Regard


In [145]:
df2 = extract_features(song_dict)

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [159]:
full_df = df.merge(df2, left_on='track_id', right_on='id')

In [160]:
full_df

,chart_appearances,track_id,title,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,65pndcjVsNEJpwxfTgcdv0,Haa Tepok,MeerFly,0.848,0.502,10,-12.368,0,0.3090,...,0.1520,0.6830,87.496,audio_features,65pndcjVsNEJpwxfTgcdv0,spotify:track:65pndcjVsNEJpwxfTgcdv0,https://api.spotify.com/v1/tracks/65pndcjVsNEJ...,https://api.spotify.com/v1/audio-analysis/65pn...,210943,4
1,1,1MRRjWwqBUyoD9dC11sZ4S,像魚,王貳浪,0.639,0.395,11,-10.053,1,0.0374,...,0.1240,0.1580,139.898,audio_features,1MRRjWwqBUyoD9dC11sZ4S,spotify:track:1MRRjWwqBUyoD9dC11sZ4S,https://api.spotify.com/v1/tracks/1MRRjWwqBUyo...,https://api.spotify.com/v1/audio-analysis/1MRR...,285272,4
2,1,7aiClxsDWFRQ0Kzk5KI5ku,hot girl bummer,blackbear,0.778,0.559,6,-7.109,0,0.0776,...,0.3990,0.6780,129.989,audio_features,7aiClxsDWFRQ0Kzk5KI5ku,spotify:track:7aiClxsDWFRQ0Kzk5KI5ku,https://api.spotify.com/v1/tracks/7aiClxsDWFRQ...,https://api.spotify.com/v1/audio-analysis/7aiC...,185093,1
3,1,3IP4qSMfGeqI2XqHFf25sj,芒种,音阙诗听,0.781,0.656,8,-5.862,1,0.0310,...,0.1140,0.5480,99.981,audio_features,3IP4qSMfGeqI2XqHFf25sj,spotify:track:3IP4qSMfGeqI2XqHFf25sj,https://api.spotify.com/v1/tracks/3IP4qSMfGeqI...,https://api.spotify.com/v1/audio-analysis/3IP4...,216000,4
4,1,5uWDUe75h2FHLZyMX9r5sp,Sunkissed,khai dreams,0.768,0.354,6,-9.484,1,0.0539,...,0.3990,0.7930,153.970,audio_features,5uWDUe75h2FHLZyMX9r5sp,spotify:track:5uWDUe75h2FHLZyMX9r5sp,https://api.spotify.com/v1/tracks/5uWDUe75h2FH...,https://api.spotify.com/v1/audio-analysis/5uWD...,126464,4
5,1,56pvqFKGXPjubh5eY6sOlv,Is There Still Anything That Love Can Do?,RADWIMPS,0.529,0.403,9,-6.380,1,0.0336,...,0.2930,0.2450,141.780,audio_features,56pvqFKGXPjubh5eY6sOlv,spotify:track:56pvqFKGXPjubh5eY6sOlv,https://api.spotify.com/v1/tracks/56pvqFKGXPju...,https://api.spotify.com/v1/audio-analysis/56pv...,414467,4
6,1,2cE5PbW9PhPmnyETXqH3VE,Grand Escape (Movie Edit) [feat. Toko Miura],RADWIMPS,0.462,0.424,4,-9.161,1,0.0367,...,0.1370,0.5420,146.061,audio_features,2cE5PbW9PhPmnyETXqH3VE,spotify:track:2cE5PbW9PhPmnyETXqH3VE,https://api.spotify.com/v1/tracks/2cE5PbW9PhPm...,https://api.spotify.com/v1/audio-analysis/2cE5...,188067,4
7,1,0TMGV2zMZXOILVtQLtkI3G,What Am I,Why Don't We,0.614,0.647,0,-4.681,0,0.0524,...,0.1040,0.3680,163.862,audio_features,0TMGV2zMZXOILVtQLtkI3G,spotify:track:0TMGV2zMZXOILVtQLtkI3G,https://api.spotify.com/v1/tracks/0TMGV2zMZXOI...,https://api.spotify.com/v1/audio-analysis/0TMG...,184665,4
8,1,1rgnBhdG2JDFTbYkYRZAku,Dance Monkey,Tones and I,0.825,0.593,6,-6.401,0,0.0988,...,0.1700,0.5400,98.078,audio_features,1rgnBhdG2JDFTbYkYRZAku,spotify:track:1rgnBhdG2JDFTbYkYRZAku,https://api.spotify.com/v1/tracks/1rgnBhdG2JDF...,https://api.spotify.com/v1/audio-analysis/1rgn...,209755,4
9,1,2tnVG71enUj33Ic2nFN6kZ,Ride It,Regard,0.893,0.748,7,-4.299,0,0.0821,...,0.0930,0.8670,117.958,audio_features,2tnVG71enUj33Ic2nFN6kZ,spotify:track:2tnVG71enUj33Ic2nFN6kZ,https://api.spotify.com/v1/tracks/2tnVG71enUj3...,https://api.spotify.com/v1/audio-analysis/2tnV...,157605,4


In [184]:
# Test all functions

# Get full year's worth of tracks on viral charts

song_dict = spotify_viral(start_date = '2018-10-02', end_date = '2019-10-01')


# Get first dataframe from results

df = pd.DataFrame(song_dict.items(), columns = ['tup', 'chart_appearances'])

# Because the initial dataframe provides a tuple of multiple variables 
#in the first column called 'tup', we recast the tuple as list so that 
# each value has its own column

df[['track_id', 'title', 'artist']] = pd.DataFrame(df.iloc[:,0].tolist())

# Drop the 'tup' column
df.drop(columns = 'tup', inplace = True)

# Get track attributes as a second dataframe 

df2 = extract_features(song_dict)

# Merge dataframes

full_df = df.merge(df2, left_on='track_id', right_on='id')



2018-10-02 00:00:00
2018-10-03 00:00:00
2018-10-04 00:00:00
2018-10-05 00:00:00
2018-10-06 00:00:00
2018-10-07 00:00:00
2018-10-08 00:00:00
2018-10-09 00:00:00
2018-10-10 00:00:00
2018-10-11 00:00:00
2018-10-12 00:00:00
2018-10-13 00:00:00
2018-10-14 00:00:00
2018-10-15 00:00:00
2018-10-16 00:00:00
2018-10-17 00:00:00
2018-10-18 00:00:00
2018-10-19 00:00:00
2018-10-20 00:00:00
2018-10-21 00:00:00
2018-10-22 00:00:00
2018-10-23 00:00:00
2018-10-24 00:00:00
2018-10-25 00:00:00
2018-10-26 00:00:00
2018-10-27 00:00:00
2018-10-28 00:00:00
2018-10-29 00:00:00
2018-10-30 00:00:00
2018-10-31 00:00:00
2018-11-01 00:00:00
2018-11-02 00:00:00
2018-11-03 00:00:00
2018-11-04 00:00:00
2018-11-05 00:00:00
2018-11-06 00:00:00
2018-11-07 00:00:00
2018-11-08 00:00:00
2018-11-09 00:00:00
2018-11-10 00:00:00
2018-11-11 00:00:00
2018-11-12 00:00:00
2018-11-13 00:00:00
2018-11-14 00:00:00
2018-11-15 00:00:00
2018-11-16 00:00:00
2018-11-17 00:00:00
2018-11-18 00:00:00
2018-11-19 00:00:00
2018-11-20 00:00:00


In [192]:
full_df.to_csv('spotify_tracks.csv', index = False)

,chart_appearances,track_id,title,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,13,4D5Ode2OfZMxIwQyvmhUe8,Lil’ Touch,Girls' Generation-Oh!GG,0.791,0.789,11,-4.694,0,0.1040,...,0.301,0.669,112.110,audio_features,4D5Ode2OfZMxIwQyvmhUe8,spotify:track:4D5Ode2OfZMxIwQyvmhUe8,https://api.spotify.com/v1/tracks/4D5Ode2OfZMx...,https://api.spotify.com/v1/audio-analysis/4D5O...,189846,4
1,24,2JqnpexlO9dmvjUMCaLCLJ,Love Someone,Lukas Graham,0.550,0.415,9,-6.557,0,0.0520,...,0.108,0.274,172.065,audio_features,2JqnpexlO9dmvjUMCaLCLJ,spotify:track:2JqnpexlO9dmvjUMCaLCLJ,https://api.spotify.com/v1/tracks/2JqnpexlO9dm...,https://api.spotify.com/v1/audio-analysis/2Jqn...,205463,4
2,20,5N4erncE7kuUccm7zEmwzk,Electricity (with Dua Lipa),Silk City,0.588,0.670,0,-6.439,1,0.0473,...,0.338,0.505,118.159,audio_features,5N4erncE7kuUccm7zEmwzk,spotify:track:5N4erncE7kuUccm7zEmwzk,https://api.spotify.com/v1/tracks/5N4erncE7kuU...,https://api.spotify.com/v1/audio-analysis/5N4e...,238173,4
3,177,25sgk305KZfyuqVBQIahim,Sweet but Psycho,Ava Max,0.719,0.704,1,-4.724,1,0.0476,...,0.166,0.628,133.002,audio_features,25sgk305KZfyuqVBQIahim,spotify:track:25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4
4,40,2dpaYNEQHiRxtZbfNsse99,Happier,Marshmello,0.687,0.792,5,-2.749,1,0.0452,...,0.167,0.671,100.015,audio_features,2dpaYNEQHiRxtZbfNsse99,spotify:track:2dpaYNEQHiRxtZbfNsse99,https://api.spotify.com/v1/tracks/2dpaYNEQHiRx...,https://api.spotify.com/v1/audio-analysis/2dpa...,214290,4
